In [357]:
import pandas as pd
import os
import glob
import json

mkdir = lambda p: 0 if os.path.exists(p) else (os.mkdir(p), 1)[1]


## DATA Paths & Constants

In [358]:
# DATA_PATH = "/home/jeevan/Jeevan_K/Projects/Pnoi-phone/Pnoi_Corpus/PNOI_CORPUS_2023/DATA"
DATA_PATH = "DATA"

GLOB_PATH = f"{DATA_PATH}/pnoistor_*/*/*"

REPORT_PATH = "./report"; mkdir(REPORT_PATH)

0

In [359]:
FILE_PARTS = ["app_version_code", "subject_ID", "file_class", "file_ID", "comment", "file_format"]
SEP="-"
META_SEP="_"

# ALL Files Dataframe

In [360]:
all_data_files = glob.glob(GLOB_PATH)


def file_info(file_path: str) -> dict:

    file_name: str = os.path.basename(file_path)
    file_name_parts = file_name.replace(".", SEP).split(sep=SEP)
    file_name_parts += [file_name, file_path]
    part_names = FILE_PARTS +["file_name", "file_path"]

    file_info_dict = { p:file_name_parts[i] for i, p in enumerate(part_names) }

    return file_info_dict

all_data_files_DF = pd.DataFrame([file_info(f) for f in all_data_files])
all_data_files_DF.to_csv(f"{REPORT_PATH}/all_data_files.csv", index=False); all_data_files_DF

,app_version_code,subject_ID,file_class,file_ID,comment,file_format,file_name,file_path
0,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
1,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
2,pnoistor_dec01,sagarkumar_b77e6507,PFT_before,720f,comnt,pdf,pnoistor_dec01-sagarkumar_b77e6507-PFT_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
3,pnoistor_dec01,sagarkumar_b77e6507,META,bdbe,comnt,json,pnoistor_dec01-sagarkumar_b77e6507-META-bdbe-c...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
4,pnoistor_dec01,vijayaomkar_b866b679,VBA_before,1970,comnt,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...
...,...,...,...,...,...,...,...,...
273,pnoistor_may2023,valli_82ee8284,LBA_before_LL,4b19,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...
274,pnoistor_may2023,valli_82ee8284,LBA_before_LU,6a5c,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...
275,pnoistor_may2023,valli_82ee8284,PFT_before,4748,comnt,pdf,pnoistor_may2023-valli_82ee8284-PFT_before-474...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...
276,pnoistor_may2023,valli_82ee8284,META,18a6,comnt,json,pnoistor_may2023-valli_82ee8284-META-18a6-comn...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...


## METADATA Column

In [361]:
filt = all_data_files_DF.loc[:, FILE_PARTS[2]].str.contains("META")
all_META_file_data_DF = all_data_files_DF[filt]; all_META_file_data_DF
all_META_file_data_DF.columns = all_data_files_DF.columns.str.replace("file", "META_file")

def read_metadata(row):
    file_path = row['META_file_path']

    file_info = { "META_file_path": file_path }

    with open(file_path, 'r') as f:
        data = json.load(f)
        bio_data = data["subjectBiodata"] | file_info
        survey_data = data["subjectSurvey"]["answeredQs"]
        survey_data = pd.DataFrame([{ "qn": q["qno"], "Q": q["question"], "A": q["answer"] } for q in survey_data])
        return (bio_data, survey_data)
#TEST
read_metadata(all_META_file_data_DF.iloc[0])


({'subjectGender': 'Male',
  'subjectName': 'Sagar Kumar ',
  'subjectSectionDone': True,
  'subjectAge': '25',
  'firebaseId': 'sagarkumar_b77e6507',
  'subjectType': 'Control',
  'subjectHeight': '165',
  'subjectRemunerationType': 'GooglePay',
  'subjectRemunerationDetails': '7042437831',
  'subjectWeight': '84',
  'META_file_path': 'DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoistor_dec01-sagarkumar_b77e6507-META-bdbe-comnt.json'},
     qn                                                  Q           A
 0    1                       What is your smoking status?  Non-smoker
 1    2            Do you have repeated episodes of cough?          No
 2    5               Have you been diagnosed with Asthma?          No
 3   19        Have you suffered from lung TB in the past?          No
 4   20         Do you have any other respiratory illness?          No
 5   22       Are you a known case of high blood pressure?          No
 6   24                  Are you a known case of diabetes?       

In [362]:
all_META_data_DF = pd.DataFrame([read_metadata(row)[0] for _, row in all_META_file_data_DF.iterrows()])
all_META_data_DF = all_META_data_DF.drop(columns=["subjectSectionDone", "subjectRemunerationType", "subjectRemunerationDetails"])
all_META_data_DF = all_META_data_DF.rename(columns={"firebaseId": FILE_PARTS[1]})
all_META_data_DF.to_csv(f"{REPORT_PATH}/all_META_data.csv", index=False); all_META_data_DF

all_META_data_DF["subjectGender"].value_counts()

Male      15
Female    14
Name: subjectGender, dtype: int64

## ALL files Dataframe 2 (With METADATA column)

In [363]:
all_data_files_DF2 = all_data_files_DF.merge(all_META_data_DF, how="left", on=[FILE_PARTS[1]]); 
all_data_files_DF2.to_csv(f"{REPORT_PATH}/all_data_files2.csv", index=False); all_data_files_DF2

,app_version_code,subject_ID,file_class,file_ID,comment,file_format,file_name,file_path,subjectGender,subjectName,subjectAge,subjectType,subjectHeight,subjectWeight,META_file_path
0,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,25,Control,165,84,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
1,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,25,Control,165,84,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
2,pnoistor_dec01,sagarkumar_b77e6507,PFT_before,720f,comnt,pdf,pnoistor_dec01-sagarkumar_b77e6507-PFT_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,25,Control,165,84,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
3,pnoistor_dec01,sagarkumar_b77e6507,META,bdbe,comnt,json,pnoistor_dec01-sagarkumar_b77e6507-META-bdbe-c...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,25,Control,165,84,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
4,pnoistor_dec01,vijayaomkar_b866b679,VBA_before,1970,comnt,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...,Female,Vijaya Omkar,64,Patient,153,59,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,pnoistor_may2023,valli_82ee8284,LBA_before_LL,4b19,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,33,Patient,149,42,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...
274,pnoistor_may2023,valli_82ee8284,LBA_before_LU,6a5c,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,33,Patient,149,42,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...
275,pnoistor_may2023,valli_82ee8284,PFT_before,4748,comnt,pdf,pnoistor_may2023-valli_82ee8284-PFT_before-474...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,33,Patient,149,42,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...
276,pnoistor_may2023,valli_82ee8284,META,18a6,comnt,json,pnoistor_may2023-valli_82ee8284-META-18a6-comn...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,33,Patient,149,42,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...


In [364]:
pd.unique(all_data_files_DF2.loc[:, "file_class"])

array(['VBA_before', 'PFT_before', 'META', 'VBA_after', 'LBA_after_LU',
       'LBA_before_LL', 'LBA_before_RL', 'PFT_after', 'LBA_after_RU',
       'LBA_before_RU', 'LBA_after_RL', 'LBA_after_LL', 'LBA_before_LU'],
      dtype=object)

## PFT Value Column

### WHEN: Before Inhaler

In [365]:
WHEN = "before"
CLASS = "PFT"
FORMAT = "tsv|csv"
filt = all_data_files_DF.loc[:, FILE_PARTS[2]].str.contains(f"{CLASS}_{WHEN}") 
filt &= all_data_files_DF.loc[:, FILE_PARTS[5]].str.contains(FORMAT) 
all_PFT_file_data_DF = all_data_files_DF[filt]
all_PFT_file_data_DF.columns = all_data_files_DF.columns.str.replace("file", f"{CLASS}_{WHEN}_file"); all_PFT_file_data_DF


,app_version_code,subject_ID,PFT_before_file_class,PFT_before_file_ID,comment,PFT_before_file_format,PFT_before_file_name,PFT_before_file_path
48,pnoistor_apr2023,anitajoby_8bf5d711,PFT_before,7782,comnt,tsv,pnoistor_apr2023-anitajoby_8bf5d711-PFT_before...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...


In [366]:
def read_pft_data(row):
    file_path = row[f'{CLASS}_{WHEN}_file_path']

    df = pd.read_csv(file_path, sep="\t")
    file_info = {
        f"{CLASS}_{WHEN}_file_path": file_path,
        "subject_ID": row["subject_ID"],
    }
    pft_dict = {}
    for i, row in df.iterrows():
        for col in df.columns[1:]:
            val_typ = row["PFT"]
            pft_key = f"{col}_{val_typ}_{WHEN}"
            pft_val = df.loc[i, col]
            pft_dict[pft_key] = pft_val

    pft_dict = pft_dict | file_info

    # df_reshape =  pd.DataFrame([pft_dict])
    return pft_dict

# Test
read_pft_data(all_PFT_file_data_DF.iloc[0])


{'FEV1_ref_before': 2.63,
 'FVC_ref_before': 3.13,
 'ratio_ref_before': 84,
 'FEV1_val_before': 3.13,
 'FVC_val_before': 3.4,
 'ratio_val_before': 92,
 'PFT_before_file_path': 'DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnoistor_apr2023-anitajoby_8bf5d711-PFT_before-7782-comnt.tsv',
 'subject_ID': 'anitajoby_8bf5d711'}

In [367]:
all_PFT_data_DF = pd.DataFrame([read_pft_data(row) for _, row in all_PFT_file_data_DF.iterrows()])
all_PFT_data_DF.to_csv(f"{REPORT_PATH}/all_{CLASS}_{WHEN}_data.csv", index=False); all_PFT_data_DF

all_PFT_data_DF

,FEV1_ref_before,FVC_ref_before,ratio_ref_before,FEV1_val_before,FVC_val_before,ratio_val_before,PFT_before_file_path,subject_ID
0,2.63,3.13,84,3.13,3.4,92,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,anitajoby_8bf5d711


#### Intermediate ALL File DF3

In [368]:
all_data_files_DF3_1 = all_data_files_DF2.merge(all_PFT_data_DF, how="left", on=[FILE_PARTS[1]])
all_data_files_DF3_1


,app_version_code,subject_ID,file_class,file_ID,comment,file_format,file_name,file_path,subjectGender,subjectName,...,subjectHeight,subjectWeight,META_file_path,FEV1_ref_before,FVC_ref_before,ratio_ref_before,FEV1_val_before,FVC_val_before,ratio_val_before,PFT_before_file_path
0,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,165,84,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,165,84,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pnoistor_dec01,sagarkumar_b77e6507,PFT_before,720f,comnt,pdf,pnoistor_dec01-sagarkumar_b77e6507-PFT_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,165,84,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pnoistor_dec01,sagarkumar_b77e6507,META,bdbe,comnt,json,pnoistor_dec01-sagarkumar_b77e6507-META-bdbe-c...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,165,84,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pnoistor_dec01,vijayaomkar_b866b679,VBA_before,1970,comnt,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...,Female,Vijaya Omkar,...,153,59,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,pnoistor_may2023,valli_82ee8284,LBA_before_LL,4b19,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,149,42,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,pnoistor_may2023,valli_82ee8284,LBA_before_LU,6a5c,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,149,42,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,pnoistor_may2023,valli_82ee8284,PFT_before,4748,comnt,pdf,pnoistor_may2023-valli_82ee8284-PFT_before-474...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,149,42,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276,pnoistor_may2023,valli_82ee8284,META,18a6,comnt,json,pnoistor_may2023-valli_82ee8284-META-18a6-comn...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,149,42,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### WHEN: After Inhaler

In [369]:
WHEN = "after"
CLASS = "PFT"
FORMAT = "tsv|csv"
filt = all_data_files_DF.loc[:, FILE_PARTS[2]].str.contains(f"{CLASS}_{WHEN}") 
filt &= all_data_files_DF.loc[:, FILE_PARTS[5]].str.contains("tsv") 
all_PFT_file_data_DF = all_data_files_DF[filt]
all_PFT_file_data_DF.columns = all_PFT_file_data_DF.columns.str.replace("file", f"{CLASS}_{WHEN}_file"); all_PFT_file_data_DF

all_PFT_data_DF = pd.DataFrame([read_pft_data(row) for _, row in all_PFT_file_data_DF.iterrows()])
all_PFT_data_DF.to_csv(f"{REPORT_PATH}/all_{CLASS}_{WHEN}_data.csv", index=False); all_PFT_data_DF

,FEV1_ref_after,FVC_ref_after,ratio_ref_after,FEV1_val_after,FVC_val_after,ratio_val_after,PFT_after_file_path,subject_ID
0,2.63,3.13,84,2.88,3.16,91,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,anitajoby_8bf5d711


In [370]:
all_data_files_DF3_2 = all_data_files_DF3_1.merge(all_PFT_data_DF, how="left", on=[FILE_PARTS[1]])

all_data_files_DF3 = all_data_files_DF3_2

all_data_files_DF3[all_data_files_DF3_2["subject_ID"].str.contains("anita")].head().columns


Index(['app_version_code', 'subject_ID', 'file_class', 'file_ID', 'comment',
       'file_format', 'file_name', 'file_path', 'subjectGender', 'subjectName',
       'subjectAge', 'subjectType', 'subjectHeight', 'subjectWeight',
       'META_file_path', 'FEV1_ref_before', 'FVC_ref_before',
       'ratio_ref_before', 'FEV1_val_before', 'FVC_val_before',
       'ratio_val_before', 'PFT_before_file_path', 'FEV1_ref_after',
       'FVC_ref_after', 'ratio_ref_after', 'FEV1_val_after', 'FVC_val_after',
       'ratio_val_after', 'PFT_after_file_path'],
      dtype='object')

## ALL files Dataframe 3 (With PFT [before + after] column)

In [371]:
all_data_files_DF3.to_csv(f"{REPORT_PATH}/all_data_files3.csv", index=False); all_data_files_DF3


,app_version_code,subject_ID,file_class,file_ID,comment,file_format,file_name,file_path,subjectGender,subjectName,...,FVC_val_before,ratio_val_before,PFT_before_file_path,FEV1_ref_after,FVC_ref_after,ratio_ref_after,FEV1_val_after,FVC_val_after,ratio_val_after,PFT_after_file_path
0,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pnoistor_dec01,sagarkumar_b77e6507,PFT_before,720f,comnt,pdf,pnoistor_dec01-sagarkumar_b77e6507-PFT_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pnoistor_dec01,sagarkumar_b77e6507,META,bdbe,comnt,json,pnoistor_dec01-sagarkumar_b77e6507-META-bdbe-c...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pnoistor_dec01,vijayaomkar_b866b679,VBA_before,1970,comnt,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...,Female,Vijaya Omkar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,pnoistor_may2023,valli_82ee8284,LBA_before_LL,4b19,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274,pnoistor_may2023,valli_82ee8284,LBA_before_LU,6a5c,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275,pnoistor_may2023,valli_82ee8284,PFT_before,4748,comnt,pdf,pnoistor_may2023-valli_82ee8284-PFT_before-474...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276,pnoistor_may2023,valli_82ee8284,META,18a6,comnt,json,pnoistor_may2023-valli_82ee8284-META-18a6-comn...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [372]:
all_data_files_DF3.columns

Index(['app_version_code', 'subject_ID', 'file_class', 'file_ID', 'comment',
       'file_format', 'file_name', 'file_path', 'subjectGender', 'subjectName',
       'subjectAge', 'subjectType', 'subjectHeight', 'subjectWeight',
       'META_file_path', 'FEV1_ref_before', 'FVC_ref_before',
       'ratio_ref_before', 'FEV1_val_before', 'FVC_val_before',
       'ratio_val_before', 'PFT_before_file_path', 'FEV1_ref_after',
       'FVC_ref_after', 'ratio_ref_after', 'FEV1_val_after', 'FVC_val_after',
       'ratio_val_after', 'PFT_after_file_path'],
      dtype='object')

## VBA File Column

### WHEN: Before

In [373]:
WHEN = "before"
CLASS = "VBA"
FORMAT = "wav|WAV"
TYPE = "aud"
filt = all_data_files_DF.loc[:, FILE_PARTS[2]].str.contains(f"{CLASS}_{WHEN}") 
filt &= all_data_files_DF.loc[:, FILE_PARTS[5]].str.contains(FORMAT) 
all_VBA_file_data_DF = all_data_files_DF[filt]
all_VBA_file_data_DF.columns = all_VBA_file_data_DF.columns.str.replace("file", f"{CLASS}_{WHEN}_{TYPE}_file")
audio_df = all_VBA_file_data_DF; audio_df.head()

,app_version_code,subject_ID,VBA_before_aud_file_class,VBA_before_aud_file_ID,comment,VBA_before_aud_file_format,VBA_before_aud_file_name,VBA_before_aud_file_path
0,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
6,pnoistor_dec01,vijayaomkar_b866b679,VBA_before,1970,comnt,wav,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...
10,pnoistor_dec01,saikeerthanaarun_3364bc1a,VBA_before,cb51,comnt,wav,pnoistor_dec01-saikeerthanaarun_3364bc1a-VBA_b...,DATA/pnoistor_dec01/saikeerthanaarun_3364bc1a/...
15,pnoistor_dec01,kumarchowdam_53f32e31,VBA_before,91bc,comnt,wav,pnoistor_dec01-kumarchowdam_53f32e31-VBA_befor...,DATA/pnoistor_dec01/kumarchowdam_53f32e31/pnoi...
16,pnoistor_dec01,sannashoukat_5213fe84,VBA_before,ad42,comnt,wav,pnoistor_dec01-sannashoukat_5213fe84-VBA_befor...,DATA/pnoistor_dec01/sannashoukat_5213fe84/pnoi...


In [374]:
FORMAT = "txt"
TYPE = "anot"
filt = all_data_files_DF.loc[:, FILE_PARTS[2]].str.contains(f"{CLASS}_{WHEN}") 
filt &= all_data_files_DF.loc[:, FILE_PARTS[5]].str.contains(FORMAT) 
all_VBA_file_data_DF = all_data_files_DF[filt]
all_VBA_file_data_DF.columns = all_VBA_file_data_DF.columns.str.replace("file", f"{CLASS}_{WHEN}_{TYPE}_file")
anot_df = all_VBA_file_data_DF; anot_df.head()

,app_version_code,subject_ID,VBA_before_anot_file_class,VBA_before_anot_file_ID,comment,VBA_before_anot_file_format,VBA_before_anot_file_name,VBA_before_anot_file_path
1,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
4,pnoistor_dec01,vijayaomkar_b866b679,VBA_before,1970,comnt,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...
11,pnoistor_dec01,saikeerthanaarun_3364bc1a,VBA_before,cb51,comnt,txt,pnoistor_dec01-saikeerthanaarun_3364bc1a-VBA_b...,DATA/pnoistor_dec01/saikeerthanaarun_3364bc1a/...
13,pnoistor_dec01,kumarchowdam_53f32e31,VBA_before,91bc,comnt,txt,pnoistor_dec01-kumarchowdam_53f32e31-VBA_befor...,DATA/pnoistor_dec01/kumarchowdam_53f32e31/pnoi...
19,pnoistor_dec01,sannashoukat_5213fe84,VBA_before,ad42,comnt,txt,pnoistor_dec01-sannashoukat_5213fe84-VBA_befor...,DATA/pnoistor_dec01/sannashoukat_5213fe84/pnoi...


In [375]:
all_VBA_anot_merge_data_DF = audio_df.merge(anot_df, how="left", on=[FILE_PARTS[0], FILE_PARTS[1], FILE_PARTS[4]])
all_VBA_anot_merge_data_DF = all_VBA_anot_merge_data_DF.drop(columns=[FILE_PARTS[0], FILE_PARTS[4]])
all_data_files_DF4_1 = all_data_files_DF3.merge(all_VBA_anot_merge_data_DF, how="left", on=[FILE_PARTS[1]])
all_data_files_DF4_1

,app_version_code,subject_ID,file_class,file_ID,comment,file_format,file_name,file_path,subjectGender,subjectName,...,VBA_before_aud_file_class,VBA_before_aud_file_ID,VBA_before_aud_file_format,VBA_before_aud_file_name,VBA_before_aud_file_path,VBA_before_anot_file_class,VBA_before_anot_file_ID,VBA_before_anot_file_format,VBA_before_anot_file_name,VBA_before_anot_file_path
0,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,VBA_before,f8bd,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,VBA_before,f8bd,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
1,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,VBA_before,f8bd,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,VBA_before,f8bd,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
2,pnoistor_dec01,sagarkumar_b77e6507,PFT_before,720f,comnt,pdf,pnoistor_dec01-sagarkumar_b77e6507-PFT_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,VBA_before,f8bd,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,VBA_before,f8bd,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
3,pnoistor_dec01,sagarkumar_b77e6507,META,bdbe,comnt,json,pnoistor_dec01-sagarkumar_b77e6507-META-bdbe-c...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,VBA_before,f8bd,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,VBA_before,f8bd,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...
4,pnoistor_dec01,vijayaomkar_b866b679,VBA_before,1970,comnt,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...,Female,Vijaya Omkar,...,VBA_before,1970,wav,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...,VBA_before,1970,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,pnoistor_may2023,valli_82ee8284,LBA_before_LL,4b19,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,VBA_before,c74e,WAV,pnoistor_may2023-valli_82ee8284-VBA_before-c74...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN
274,pnoistor_may2023,valli_82ee8284,LBA_before_LU,6a5c,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,VBA_before,c74e,WAV,pnoistor_may2023-valli_82ee8284-VBA_before-c74...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN
275,pnoistor_may2023,valli_82ee8284,PFT_before,4748,comnt,pdf,pnoistor_may2023-valli_82ee8284-PFT_before-474...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,VBA_before,c74e,WAV,pnoistor_may2023-valli_82ee8284-VBA_before-c74...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN
276,pnoistor_may2023,valli_82ee8284,META,18a6,comnt,json,pnoistor_may2023-valli_82ee8284-META-18a6-comn...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,VBA_before,c74e,WAV,pnoistor_may2023-valli_82ee8284-VBA_before-c74...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN


In [376]:
all_data_files_DF4_1.columns

Index(['app_version_code', 'subject_ID', 'file_class', 'file_ID', 'comment',
       'file_format', 'file_name', 'file_path', 'subjectGender', 'subjectName',
       'subjectAge', 'subjectType', 'subjectHeight', 'subjectWeight',
       'META_file_path', 'FEV1_ref_before', 'FVC_ref_before',
       'ratio_ref_before', 'FEV1_val_before', 'FVC_val_before',
       'ratio_val_before', 'PFT_before_file_path', 'FEV1_ref_after',
       'FVC_ref_after', 'ratio_ref_after', 'FEV1_val_after', 'FVC_val_after',
       'ratio_val_after', 'PFT_after_file_path', 'VBA_before_aud_file_class',
       'VBA_before_aud_file_ID', 'VBA_before_aud_file_format',
       'VBA_before_aud_file_name', 'VBA_before_aud_file_path',
       'VBA_before_anot_file_class', 'VBA_before_anot_file_ID',
       'VBA_before_anot_file_format', 'VBA_before_anot_file_name',
       'VBA_before_anot_file_path'],
      dtype='object')

In [377]:
WHEN = "after"
CLASS = "VBA"
FORMAT = "wav|WAV"
TYPE = "aud"
filt = all_data_files_DF.loc[:, FILE_PARTS[2]].str.contains(f"{CLASS}_{WHEN}") 
filt &= all_data_files_DF.loc[:, FILE_PARTS[5]].str.contains(FORMAT) 
all_VBA_file_data_DF = all_data_files_DF[filt]
all_VBA_file_data_DF.columns = all_VBA_file_data_DF.columns.str.replace("file", f"{CLASS}_{WHEN}_{TYPE}_file")
audio_df = all_VBA_file_data_DF; audio_df.head()

,app_version_code,subject_ID,VBA_after_aud_file_class,VBA_after_aud_file_ID,comment,VBA_after_aud_file_format,VBA_after_aud_file_name,VBA_after_aud_file_path
8,pnoistor_dec01,vijayaomkar_b866b679,VBA_after,4497,comnt,wav,pnoistor_dec01-vijayaomkar_b866b679-VBA_after-...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...
33,pnoistor_apr2023,rayniva_f3e542e1,VBA_after,a71d,comnt,WAV,pnoistor_apr2023-rayniva_f3e542e1-VBA_after-a7...,DATA/pnoistor_apr2023/rayniva_f3e542e1/pnoisto...
65,pnoistor_apr2023,anitajoby_8bf5d711,VBA_after,a97f,comnt,WAV,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...
85,pnoistor_apr2023,sujatan_bdd161b6,VBA_after,f976,comnt,WAV,pnoistor_apr2023-sujatan_bdd161b6-VBA_after-f9...,DATA/pnoistor_apr2023/sujatan_bdd161b6/pnoisto...
94,pnoistor_apr2023,royjoseph_4136005e,VBA_after,b2f2,comnt,WAV,pnoistor_apr2023-royjoseph_4136005e-VBA_after-...,DATA/pnoistor_apr2023/royjoseph_4136005e/pnois...


In [378]:
FORMAT = "txt"
TYPE = "anot"
filt = all_data_files_DF.loc[:, FILE_PARTS[2]].str.contains(f"{CLASS}_{WHEN}") 
filt &= all_data_files_DF.loc[:, FILE_PARTS[5]].str.contains(FORMAT) 
all_VBA_file_data_DF = all_data_files_DF[filt]
all_VBA_file_data_DF.columns = all_VBA_file_data_DF.columns.str.replace("file", f"{CLASS}_{WHEN}_{TYPE}_file")
anot_df = all_VBA_file_data_DF; anot_df.head()

,app_version_code,subject_ID,VBA_after_anot_file_class,VBA_after_anot_file_ID,comment,VBA_after_anot_file_format,VBA_after_anot_file_name,VBA_after_anot_file_path
5,pnoistor_dec01,vijayaomkar_b866b679,VBA_after,4497,comnt,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_after-...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...
36,pnoistor_apr2023,rayniva_f3e542e1,VBA_after,a71d,comnt,txt,pnoistor_apr2023-rayniva_f3e542e1-VBA_after-a7...,DATA/pnoistor_apr2023/rayniva_f3e542e1/pnoisto...
51,pnoistor_apr2023,anitajoby_8bf5d711,VBA_after,a97f,comnt,txt,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...
79,pnoistor_apr2023,sujatan_bdd161b6,VBA_after,f976,comnt,txt,pnoistor_apr2023-sujatan_bdd161b6-VBA_after-f9...,DATA/pnoistor_apr2023/sujatan_bdd161b6/pnoisto...
109,pnoistor_apr2023,royjoseph_4136005e,VBA_after,b2f2,comnt,txt,pnoistor_apr2023-royjoseph_4136005e-VBA_after-...,DATA/pnoistor_apr2023/royjoseph_4136005e/pnois...


In [379]:
all_VBA_anot_merge_data_DF = audio_df.merge(anot_df, how="left", on=[FILE_PARTS[0], FILE_PARTS[1], FILE_PARTS[4]])
all_VBA_anot_merge_data_DF = all_VBA_anot_merge_data_DF.drop(columns=[FILE_PARTS[0], FILE_PARTS[4]])
all_data_files_DF4_2 = all_data_files_DF4_1.merge(all_VBA_anot_merge_data_DF, how="left", on=[FILE_PARTS[1]])
all_data_files_DF4_2

all_data_files_DF4 = all_data_files_DF4_2

all_data_files_DF4[all_data_files_DF3_2["subject_ID"].str.contains("anita")].head()

,app_version_code,subject_ID,file_class,file_ID,comment,file_format,file_name,file_path,subjectGender,subjectName,...,VBA_after_aud_file_class,VBA_after_aud_file_ID,VBA_after_aud_file_format,VBA_after_aud_file_name,VBA_after_aud_file_path,VBA_after_anot_file_class,VBA_after_anot_file_ID,VBA_after_anot_file_format,VBA_after_anot_file_name,VBA_after_anot_file_path
44,pnoistor_apr2023,anitajoby_8bf5d711,LBA_before_LL,5214,comnt,wav,pnoistor_apr2023-anitajoby_8bf5d711-LBA_before...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,Female,Anita Joby,...,VBA_after,a97f,WAV,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,VBA_after,a97f,txt,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...
45,pnoistor_apr2023,anitajoby_8bf5d711,LBA_before_RU,5f82,comnt,txt,pnoistor_apr2023-anitajoby_8bf5d711-LBA_before...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,Female,Anita Joby,...,VBA_after,a97f,WAV,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,VBA_after,a97f,txt,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...
46,pnoistor_apr2023,anitajoby_8bf5d711,PFT_after,acf4,comnt,tsv,pnoistor_apr2023-anitajoby_8bf5d711-PFT_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,Female,Anita Joby,...,VBA_after,a97f,WAV,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,VBA_after,a97f,txt,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...
47,pnoistor_apr2023,anitajoby_8bf5d711,LBA_after_RU,2d46,comnt,wav,pnoistor_apr2023-anitajoby_8bf5d711-LBA_after_...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,Female,Anita Joby,...,VBA_after,a97f,WAV,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,VBA_after,a97f,txt,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...
48,pnoistor_apr2023,anitajoby_8bf5d711,PFT_before,7782,comnt,tsv,pnoistor_apr2023-anitajoby_8bf5d711-PFT_before...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,Female,Anita Joby,...,VBA_after,a97f,WAV,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...,VBA_after,a97f,txt,pnoistor_apr2023-anitajoby_8bf5d711-VBA_after-...,DATA/pnoistor_apr2023/anitajoby_8bf5d711/pnois...


In [380]:
all_data_files_DF4_2.columns

Index(['app_version_code', 'subject_ID', 'file_class', 'file_ID', 'comment',
       'file_format', 'file_name', 'file_path', 'subjectGender', 'subjectName',
       'subjectAge', 'subjectType', 'subjectHeight', 'subjectWeight',
       'META_file_path', 'FEV1_ref_before', 'FVC_ref_before',
       'ratio_ref_before', 'FEV1_val_before', 'FVC_val_before',
       'ratio_val_before', 'PFT_before_file_path', 'FEV1_ref_after',
       'FVC_ref_after', 'ratio_ref_after', 'FEV1_val_after', 'FVC_val_after',
       'ratio_val_after', 'PFT_after_file_path', 'VBA_before_aud_file_class',
       'VBA_before_aud_file_ID', 'VBA_before_aud_file_format',
       'VBA_before_aud_file_name', 'VBA_before_aud_file_path',
       'VBA_before_anot_file_class', 'VBA_before_anot_file_ID',
       'VBA_before_anot_file_format', 'VBA_before_anot_file_name',
       'VBA_before_anot_file_path', 'VBA_after_aud_file_class',
       'VBA_after_aud_file_ID', 'VBA_after_aud_file_format',
       'VBA_after_aud_file_name', 'VBA_a

In [381]:
all_data_files_DF4.to_csv(f"{REPORT_PATH}/all_data_files4.csv", index=False); all_data_files_DF4

,app_version_code,subject_ID,file_class,file_ID,comment,file_format,file_name,file_path,subjectGender,subjectName,...,VBA_after_aud_file_class,VBA_after_aud_file_ID,VBA_after_aud_file_format,VBA_after_aud_file_name,VBA_after_aud_file_path,VBA_after_anot_file_class,VBA_after_anot_file_ID,VBA_after_anot_file_format,VBA_after_anot_file_name,VBA_after_anot_file_path
0,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,wav,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,pnoistor_dec01,sagarkumar_b77e6507,VBA_before,f8bd,comnt,txt,pnoistor_dec01-sagarkumar_b77e6507-VBA_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pnoistor_dec01,sagarkumar_b77e6507,PFT_before,720f,comnt,pdf,pnoistor_dec01-sagarkumar_b77e6507-PFT_before-...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pnoistor_dec01,sagarkumar_b77e6507,META,bdbe,comnt,json,pnoistor_dec01-sagarkumar_b77e6507-META-bdbe-c...,DATA/pnoistor_dec01/sagarkumar_b77e6507/pnoist...,Male,Sagar Kumar,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,pnoistor_dec01,vijayaomkar_b866b679,VBA_before,1970,comnt,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_before...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...,Female,Vijaya Omkar,...,VBA_after,4497,wav,pnoistor_dec01-vijayaomkar_b866b679-VBA_after-...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...,VBA_after,4497,txt,pnoistor_dec01-vijayaomkar_b866b679-VBA_after-...,DATA/pnoistor_dec01/vijayaomkar_b866b679/pnois...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,pnoistor_may2023,valli_82ee8284,LBA_before_LL,4b19,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LL-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,VBA_after,8748,WAV,pnoistor_may2023-valli_82ee8284-VBA_after-8748...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN
274,pnoistor_may2023,valli_82ee8284,LBA_before_LU,6a5c,comnt,wav,pnoistor_may2023-valli_82ee8284-LBA_before_LU-...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,VBA_after,8748,WAV,pnoistor_may2023-valli_82ee8284-VBA_after-8748...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN
275,pnoistor_may2023,valli_82ee8284,PFT_before,4748,comnt,pdf,pnoistor_may2023-valli_82ee8284-PFT_before-474...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,VBA_after,8748,WAV,pnoistor_may2023-valli_82ee8284-VBA_after-8748...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN
276,pnoistor_may2023,valli_82ee8284,META,18a6,comnt,json,pnoistor_may2023-valli_82ee8284-META-18a6-comn...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,Female,Valli,...,VBA_after,8748,WAV,pnoistor_may2023-valli_82ee8284-VBA_after-8748...,DATA/pnoistor_may2023/valli_82ee8284/pnoistor_...,NaN,NaN,NaN,NaN,NaN


In [382]:
def dataset_report(df: pd.DataFrame):
    subjects = pd.DataFrame({ "subject_id": pd.unique(df.loc[:, FILE_PARTS[1]]) })


    return subjects

dataset_report(all_data_files_DF3)

,subject_id
0,sagarkumar_b77e6507
1,vijayaomkar_b866b679
2,saikeerthanaarun_3364bc1a
3,kumarchowdam_53f32e31
4,sannashoukat_5213fe84
5,manju_e54090da
6,rayniva_f3e542e1
7,anitajoby_8bf5d711
8,sujatan_bdd161b6
9,royjoseph_4136005e
